### Тема “Генерация текстов (языковое моделирование)”

<i>Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [2]:
import tensorflow as tf
import keras
import numpy as np
import os
import time

In [ ]:
START_CHAR = '\b'
END_CHAR = '\t'
PADDING_CHAR = '\a'
text = ''
#chars=set([START_CHAR, '\n', END_CHAR])
chars=set()

In [3]:
path_to_file = 'pushkin_stihi2.txt' #подготовил dataset

In [4]:
text = open(path_to_file, 'r', encoding='cp1251').read()
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 208312 characters


In [5]:
print(text[:500])

Сказка о попе и о работнике его Балде
Жил-был поп,
Толоконный лоб.
Пошел поп по базару
Посмотреть кой-какого товару.
Навстречу ему Балда
Идет, сам не зная куда.
"Что, батька, так рано поднялся?
Чего ты взыскался?"
Поп ему в ответ: "Нужен мне работник:
Повар, конюх и плотник.
А где найти мне такого
Служителя не слишком дорогого?"
Балда говорит: "Буду служить тебе славно,
Усердно и очень исправно,
В год за три щелка тебе по лбу,
Есть же мне давай вареную полбу".
Призадумался поп,
Стал себе почесыв


In [6]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

117 unique characters


In [7]:
vocab

['\n',
 ' ',
 '!',
 '"',
 "'",
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Щ',
 'Ы',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ъ',
 'ы',
 'ь',
 'э',
 'ю',
 'я']

In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
text_as_int, text, len(text_as_int), len(text)

(array([72, 95, 85, ..., 97,  7,  0]),
 'Сказка о попе и о работнике его Балде\nЖил-был поп,\nТолоконный лоб.\nПошел поп по базару\nПосмотреть кой-какого товару.\nНавстречу ему Балда\nИдет, сам не зная куда.\n"Что, батька, так рано поднялся?\nЧего ты взыскался?"\nПоп ему в ответ: "Нужен мне работник:\nПовар, конюх и плотник.\nА где найти мне такого\nСлужителя не слишком дорогого?"\nБалда говорит: "Буду служить тебе славно,\nУсердно и очень исправно,\nВ год за три щелка тебе по лбу,\nЕсть же мне давай вареную полбу".\nПризадумался поп,\nСтал себе почесывать лоб.\nЩелк щелку ведь розь.\nДа понадеялся он на русский авось.\nПон говорит Балде: "Ладно.\nНе будет нам обоим накладно.\nПоживи-ка на моем подворье,\nОкажи свое усердие и проворье".\nЖивет Балда в поповом доме,\nСпит себе на соломе,\nЕст за четверых,\nРаботает за семерых;\nДо светла все у него пляшет.\nЛошадь запряжет, полосу вспашет,\nПечь затопит, все заготовит, закупит,\nЯичко испечет да сам и облупит.\nПопадья Балдой не нахвали

In [10]:
text_as_int

array([72, 95, 85, ..., 97,  7,  0])

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

С
к
а
з
к
а
 
о
 
п


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Сказка о попе и о работнике его Балде\nЖил-был поп,\nТолоконный лоб.\nПошел поп по базару\nПосмотреть кой'
'-какого товару.\nНавстречу ему Балда\nИдет, сам не зная куда.\n"Что, батька, так рано поднялся?\nЧего ты '
'взыскался?"\nПоп ему в ответ: "Нужен мне работник:\nПовар, конюх и плотник.\nА где найти мне такого\nСлуж'
'ителя не слишком дорогого?"\nБалда говорит: "Буду служить тебе славно,\nУсердно и очень исправно,\nВ год'
' за три щелка тебе по лбу,\nЕсть же мне давай вареную полбу".\nПризадумался поп,\nСтал себе почесывать л'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Сказка о попе и о работнике его Балде\nЖил-был поп,\nТолоконный лоб.\nПошел поп по базару\nПосмотреть ко'
Target data: 'казка о попе и о работнике его Балде\nЖил-был поп,\nТолоконный лоб.\nПошел поп по базару\nПосмотреть кой'


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [17]:
# model = tf.keras.Sequential(
#     [
#         tf.keras.layers.Embedding(vocab_size, embedding_dim),
#         tf.keras.layers.LSTM(rnn_units, return_sequences=True),
#         tf.keras.layers.Dense(vocab_size)
#     ]
# )

#### Используем сетку GRU, она быстрее и потребляет меньше памяти.

In [42]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [43]:
model

In [27]:


# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                                   batch_input_shape=[batch_size, None]),
                                 
#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),

#          tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),
        
#         tf.keras.layers.LSTM(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),
                                   
#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model

In [ ]:
# model = build_model(
#     vocab_size=len(vocab),
#     embedding_dim=embedding_dim,
#     rnn_units=rnn_units,
#     batch_size=BATCH_SIZE)

In [44]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 117) # (batch_size, sequence_length, vocab_size)


In [45]:
model.summary()

Model: "rn_ngenerator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  14976     
                                                                 
 gru_2 (GRU)                 multiple                  3545088   
                                                                 
 gru_3 (GRU)                 multiple                  6297600   
                                                                 
 dense_3 (Dense)             multiple                  119925    
                                                                 
Total params: 9,977,589
Trainable params: 9,977,589
Non-trainable params: 0
_________________________________________________________________


In [30]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 117), dtype=float32, numpy=
array([[-5.6708278e-03, -1.8066957e-03, -4.9595926e-03, ...,
        -1.3312420e-03,  4.0864036e-04,  6.1262050e-05],
       [-2.1843009e-03, -2.9685798e-03, -1.5467792e-04, ...,
         2.9462781e-03,  2.1222015e-03,  1.6686242e-03],
       [-2.0420263e-03,  2.0102540e-03,  7.2059676e-04, ...,
         4.5581060e-03, -1.8847054e-03,  3.1061945e-03],
       ...,
       [ 3.7789948e-03,  2.6850314e-03, -1.0237403e-03, ...,
         7.8624091e-04, -1.1829364e-03,  6.3171436e-04],
       [ 2.2072953e-03, -6.1203493e-04, -4.7985115e-04, ...,
         2.7336157e-03, -3.5342351e-03,  3.3164877e-03],
       [-2.5573596e-03, -4.5903181e-03,  7.0557953e-04, ...,
         3.5373666e-03,  1.3942525e-03, -4.3521561e-03]], dtype=float32)>

In [31]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [32]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ится,\nНос ужалил богатырь:\nНа носу вскочил волдырь.\nИ опять пошла тревога:\n"Помогите, ради бога!\nКар'

Next Char Predictions: 
 'ЭЭЫhюPmВQя!zBunzУШcсМтgйРдюРГFpпъcЭиQFюЧюэHОhтsо:кДnоIнвgFMШLеc;юпyКtдidоМaГзcMвмыиCzАTбQБцwгHHВILГК'


In [33]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 117)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.762185


In [46]:
model.compile(optimizer='adam', loss=loss)

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints_gru' # Сохраняем результаты сети с GRU
#checkpoint_dir = 'training_checkpoints_lstm' # Сохраняем результату сети с LSTM
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [36]:
checkpoint_prefix

'training_checkpoints\\ckpt_{epoch}'

In [52]:
EPOCHS = 7

In [53]:
%%time
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/8
32/32 [==============================] - 509s 16s/step - loss: 3.4391
Epoch 2/8
32/32 [==============================] - 450s 14s/step - loss: 3.4238
Epoch 3/8
32/32 [==============================] - 412s 13s/step - loss: 3.4199
Epoch 4/8
32/32 [==============================] - 414s 13s/step - loss: 3.4163
Epoch 5/8
32/32 [==============================] - 414s 13s/step - loss: 3.4135
Epoch 6/8
32/32 [==============================] - 512s 16s/step - loss: 3.4102
Epoch 7/8
32/32 [==============================] - 459s 14s/step - loss: 3.4068
Epoch 8/8
32/32 [==============================] - 414s 13s/step - loss: 3.4052
Wall time: 1h 37s


In [54]:
tf.train.latest_checkpoint(checkpoint_dir)

'training_checkpoints\\ckpt_7'

In [47]:
# Загрузка обученной модели, запускать только если сохранены результаты
#
#model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # Модель с LSTM
#model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [48]:
model.summary()

Model: "rn_ngenerator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  14976     
                                                                 
 gru_2 (GRU)                 multiple                  3545088   
                                                                 
 gru_3 (GRU)                 multiple                  6297600   
                                                                 
 dense_3 (Dense)             multiple                  119925    
                                                                 
Total params: 9,977,589
Trainable params: 9,977,589
Non-trainable params: 0
_________________________________________________________________


In [49]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [56]:
text_ = generate_text(model, start_string=u"Мама мыла раму ")
print('Текст после 8 эпох')
print(text_)

Текст после 8 эпох
Мама мыла раму коле м уерсе бо с сллд олсо елоо сле и пи
пи п оуа лма илооа в ехлуи тспр ы оалдыеа
рпнпсе анмв в пе итд но са стео птсу ва о а пео е е ддо
азахи 
, с пмаа ео аа наломя се с но сли иле па,ное о и уа слев пн бо о соте с, сс има смь о оо о на с
д
и стготно сеа с а оо елко
а и
оюс о, пмно ние е ае б ео к пи
ва о вео

 г сВлоа
та,  вла о о со е бв им а оылуе,
хвлза ст етв пе Овто о т о нс мно елзссеа со са оег оуакхсе о снсно не сте
алза бк ояа, слс н рорй трете чнв
лве сПно бм па с е пи
ыо о нилсес


In [57]:
%%time
EPOCHS = 8
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/8
32/32 [==============================] - 475s 15s/step - loss: 3.4054
Epoch 2/8
32/32 [==============================] - 491s 15s/step - loss: 3.4022
Epoch 3/8
32/32 [==============================] - 474s 15s/step - loss: 3.3994
Epoch 4/8
32/32 [==============================] - 527s 17s/step - loss: 3.3970
Epoch 5/8
32/32 [==============================] - 500s 16s/step - loss: 3.3958
Epoch 6/8
32/32 [==============================] - 508s 16s/step - loss: 3.3927
Epoch 7/8
32/32 [==============================] - 480s 15s/step - loss: 3.3905
Epoch 8/8
32/32 [==============================] - 481s 15s/step - loss: 3.3925
Wall time: 1h 5min 37s


In [58]:
text_ = generate_text(model, start_string=u"Мама мыла раму ")
print('Текст после 16 эпох')
print(text_)

Текст после 16 эпох
Мама мыла раму е ог оео о пе о
д
мтиа се
лиа о е по дтгод п с вве в пг вей еа
к тло
 пза и, о блоло о са
 зи зиле нвдеы а с пе по с вто на не н с сра са е са
 омо уима но пе о ба иа
е ста
 в нетволсра ет со а сае по бе д пио е
 сао о
вна д но о са. пь а сна пуа и но пиа во ч а ичсм, бно а
а со бар мав се нпсе пимелв уе сд о ииелпов сие со е сы дрлоо пинво о па, е
 са в же пе олн,ло и зе д са о г о о
 ио ннны стеелитсе пе, нна ппо са ва ние иаена соы не о е нбстиро ея са,
о иа о о сиа
 па е сыно о о по на вм си


In [59]:
%%time
EPOCHS = 8
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/8
32/32 [==============================] - 481s 15s/step - loss: 3.3863
Epoch 2/8
32/32 [==============================] - 498s 16s/step - loss: 3.3401
Epoch 3/8
32/32 [==============================] - 523s 16s/step - loss: 3.2744
Epoch 4/8
32/32 [==============================] - 525s 16s/step - loss: 3.1891
Epoch 5/8
32/32 [==============================] - 514s 16s/step - loss: 3.0919
Epoch 6/8
32/32 [==============================] - 485s 15s/step - loss: 3.0287
Epoch 7/8
32/32 [==============================] - 524s 16s/step - loss: 2.9802
Epoch 8/8
32/32 [==============================] - 512s 16s/step - loss: 2.9361
Wall time: 1h 7min 42s


In [60]:
text_ = generate_text(model, start_string=u"Мама мыла раму ")
print('Текст после 24 эпох')
print(text_)

Текст после 24 эпох
Мама мыла раму сама орано сора та нонадо са сома о на дети одлу сна за со
, се са
та ба са и сы бо ва со ро са со
 зе пнама о ба о м ру не о мра си зиме в на е ка
 олале и со вомеда но сет не го но о те и зономолно е м зе во ге се во
,
х вы и стро у,
,
 на жтро во на, натри со о ни но
еве ана о о дке ажи го жито се у ме па со га сета кы и пе са соета ба тне,
 ме мо гай но ном стамо по ву о жаме са набе са
 не сета го а
 сово ли за ни е бора о
 по де со воныа
но вто по ме д и бе вито номо со ити тст са зо на т 
